In [1]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve

In [3]:
df_train = pd.read_csv('../../data/external/train.csv')
print(df_train.shape) # (28698, 2305)
df_train.head()

(28698, 2305)


,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,...,feature 2295,feature 2296,feature 2297,feature 2298,feature 2299,feature 2300,feature 2301,feature 2302,feature 2303,emotion
0,108,83,63,65,89,111,121,121,125,111,...,140,131,121,119,115,111,116,122,122,3
1,137,142,159,162,158,134,113,167,174,177,...,76,98,172,196,192,191,191,192,192,3
2,111,148,155,167,181,191,194,193,195,199,...,152,199,190,121,58,59,57,56,60,3
3,151,156,121,100,80,116,151,169,183,185,...,198,196,197,200,203,205,171,33,32,3
4,248,187,149,130,97,140,139,117,197,251,...,72,69,76,98,123,171,195,200,194,3


In [5]:
df_test = pd.read_csv('../../data/external/test.csv')
print(df_test.shape) # (7177, 2305)
df_test.head()

(7177, 2305)


,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,...,feature 2295,feature 2296,feature 2297,feature 2298,feature 2299,feature 2300,feature 2301,feature 2302,feature 2303,emotion
0,5,4,5,9,10,9,10,12,8,14,...,9,10,8,9,15,19,17,13,11,3
1,19,21,22,18,20,21,16,18,19,18,...,154,167,166,149,70,46,52,57,49,3
2,228,229,230,229,228,227,226,226,226,221,...,22,29,37,34,29,30,38,30,19,3
3,25,33,43,30,46,84,105,118,139,142,...,166,137,111,125,124,128,127,130,136,3
4,33,29,15,15,20,36,40,54,79,98,...,90,80,81,72,49,43,34,75,78,3


In [ ]:
emotion_labels = {0:'angry', 1:'disgust', 2:'fear', 3:'happy', 4:'sad', 5:'surprise', 6:'neutral'}

# SVM

In [6]:
X = df_train.iloc[:, 0:2304].values
y = df_train.iloc[:, -1].values
print(X.shape)
print(y.shape)

(28698, 2304)
(28698,)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1, stratify=y)

## K-fold cross-validation

In [8]:
pipe_svc = make_pipeline(MinMaxScaler(),
                         SVC(random_state=1))

In [9]:
kfold = StratifiedKFold(n_splits=10).split(X_train, y_train)

scores = []
for k, (train, test) in enumerate(kfold):
    pipe_svc.fit(X_train[train], y_train[train])
    score = pipe_svc.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: %2d, Class dist.: %s, Acc: %.3f' % (k+1,
          np.bincount(y_train[train]), score))
    
print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

Fold:  1, Class dist.: [2516  268 2581 4545 3043 1998 3128], Acc: 0.435
Fold:  2, Class dist.: [2516  268 2581 4545 3043 1998 3128], Acc: 0.423
Fold:  3, Class dist.: [2516  268 2581 4545 3043 1998 3128], Acc: 0.425
Fold:  4, Class dist.: [2516  268 2581 4545 3043 1998 3128], Acc: 0.417
Fold:  5, Class dist.: [2517  268 2581 4545 3043 1998 3127], Acc: 0.445
Fold:  6, Class dist.: [2517  268 2581 4545 3043 1998 3127], Acc: 0.425
Fold:  7, Class dist.: [2517  268 2581 4545 3043 1998 3127], Acc: 0.427
Fold:  8, Class dist.: [2517  268 2581 4545 3043 1998 3127], Acc: 0.433
Fold:  9, Class dist.: [2516  269 2582 4545 3043 1998 3127], Acc: 0.408
Fold: 10, Class dist.: [2516  269 2582 4545 3042 1998 3128], Acc: 0.417

CV accuracy: 0.425 +/- 0.010


In [ ]:
scores = cross_val_score(estimator=pipe_svc,
                         X=X_train,
                         y=y_train,
                         cv=10,
                         n_jobs=1)
print('CV accuracy scores: %s' % scores)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

## Tuning hyperparameters via grid search

In [ ]:
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

param_grid = [{'svc__C': param_range, 
               'svc__kernel': ['linear']},
              {'svc__C': param_range, 
               'svc__gamma': param_range, 
               'svc__kernel': ['rbf']}]

gs = GridSearchCV(estimator=pipe_svc, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=10,
                  n_jobs=-1)
gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

In [ ]:
clf = gs.best_estimator_
print('Test accuracy: %.3f' % clf.score(X_test, y_test))

## Algorithm selection with nested cross-validation

In [ ]:
best_gs = clf.fit(X_train, y_train) 
# gs = GridSearchCV(estimator=pipe_svc,
#                   param_grid=best_param,
#                   scoring='accuracy',
#                   cv=2)

scores = cross_val_score(best_gs, X_train, y_train, 
                         scoring='accuracy', cv=10)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),
                                      np.std(scores)))

## Diagnosing bias and variance problems with learning curves

In [ ]:
train_sizes, train_scores, test_scores =\
                learning_curve(estimator=pipe_svm,
                               X=X_train,
                               y=y_train,
                               train_sizes=np.linspace(0.1, 1.0, 10),
                               cv=10,
                               n_jobs=1)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.plot(train_sizes, train_mean,
         color='blue', marker='o',
         markersize=5, label='Training accuracy')

plt.fill_between(train_sizes,
                 train_mean + train_std,
                 train_mean - train_std,
                 alpha=0.15, color='blue')

plt.plot(train_sizes, test_mean,
         color='green', linestyle='--',
         marker='s', markersize=5,
         label='Validation accuracy')

plt.fill_between(train_sizes,
                 test_mean + test_std,
                 test_mean - test_std,
                 alpha=0.15, color='green')

plt.grid()
plt.xlabel('Number of training examples')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.8, 1.03])
plt.tight_layout()
plt.show()

## Addressing over- and underfitting with validation curves

In [ ]:
from sklearn.model_selection import validation_curve


param_range = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
train_scores, test_scores = validation_curve(
                estimator=pipe_lr, 
                X=X_train, 
                y=y_train, 
                param_name='logisticregression__C', 
                param_range=param_range,
                cv=10)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.plot(param_range, train_mean, 
         color='blue', marker='o', 
         markersize=5, label='Training accuracy')

plt.fill_between(param_range, train_mean + train_std,
                 train_mean - train_std, alpha=0.15,
                 color='blue')

plt.plot(param_range, test_mean, 
         color='green', linestyle='--', 
         marker='s', markersize=5, 
         label='Validation accuracy')

plt.fill_between(param_range, 
                 test_mean + test_std,
                 test_mean - test_std, 
                 alpha=0.15, color='green')

plt.grid()
plt.xscale('log')
plt.legend(loc='lower right')
plt.xlabel('Parameter C')
plt.ylabel('Accuracy')
plt.ylim([0.8, 1.0])
plt.tight_layout()
# plt.savefig('images/06_06.png', dpi=300)
plt.show()